# Comparing methods to simulate Ornstein-Uhlenbeck bridge 

In [26]:
import DiffusionBridge as db
import torch
import matplotlib.pyplot as plt
from DiffusionBridge.utils import normal_logpdf
plt.style.use('ggplot')

In [27]:
# problem settings
d = 1
alpha = torch.tensor(0.0)
beta = torch.tensor(2.0)
f = lambda t,x: alpha - beta * x 
sigma = torch.tensor(1.0)
T = torch.tensor(1.0)
M = 50
diffusion = db.diffusion.model(f, sigma, d, T, M)

# # initial and terminal constraints
X0 = 1.0 * torch.ones(d)
XT = 1.0 * torch.ones(d)

# transition density
ratio = alpha / beta
transition_mean = lambda t,x: ratio + (x - ratio) * torch.exp(- beta * t)
transition_var = lambda t: (1.0 - torch.exp(-2.0 * beta * t)) / (2.0 * beta)
score_transition = lambda t,x: (transition_mean(t, X0) - x) / transition_var(t)
log_transition_density = normal_logpdf(XT.reshape(1, d), transition_mean(T, X0), transition_var(T))
print('log-transition density: ' + str(float(log_transition_density)))

# marginal density
marginal_var = lambda t: 1.0 / (1.0 / transition_var(t) + torch.exp(- 2.0 * beta * (T-t)) / transition_var(T-t))
marginal_mean = lambda t: (transition_mean(t,X0) / transition_var(t) + XT * torch.exp(- beta * (T-t)) / transition_var(T-t)) * marginal_var(t) 
score_marginal = lambda t,x: (marginal_mean(t) - x) / marginal_var(t)
grad_logh = lambda t,x: (XT - transition_mean(T - t, x)) * torch.exp(- beta * (T - t)) / transition_var(T - t)

# repetitions
N = 2**10
R = 100


log-transition density: -1.739736795425415


In [28]:
# learn backward diffusion bridge process with score matching
epsilon = 1e-3
minibatch = 100
num_iterations = 500
learning_rate = 0.01
ema_momentum = 0.99
output = diffusion.learn_score_transition(X0, XT, epsilon, minibatch, num_iterations, learning_rate, ema_momentum)
score_transition_net = output['net']

# simulate backward diffusion bridge (BDB) process with approximate score
BDB = {'ess' : torch.zeros(R), 'logestimate' : torch.zeros(R), 'acceptrate' : torch.zeros(R)}
for r in range(R):
    with torch.no_grad():
        output = diffusion.simulate_bridge_backwards(score_transition_net, X0, XT, epsilon, N, modify = False)
        trajectories = output['trajectories']
        log_proposal = output['logdensity']
    log_target = diffusion.law_bridge(trajectories) 
    log_weights = log_target - log_proposal
    
    # importance sampling
    max_log_weights = torch.max(log_weights)
    weights = torch.exp(log_weights - max_log_weights)
    norm_weights = weights / torch.sum(weights)
    ess = 1.0 / torch.sum(norm_weights**2)
    log_transition_estimate = torch.log(torch.mean(weights)) + max_log_weights
    BDB['ess'][r] = ess
    BDB['logestimate'][r] = log_transition_estimate
    
    # independent Metropolis-Hastings
    initial = diffusion.simulate_bridge_backwards(score_transition_net, X0, XT, epsilon, 1, modify = False)
    current_trajectory = initial['trajectories']
    current_log_proposal = initial['logdensity'] 
    current_log_target = diffusion.law_bridge(current_trajectory)
    current_log_weight = current_log_target - current_log_proposal
    num_accept = 0
    for n in range(N):
        proposed_trajectory = trajectories[n, :, :]
        proposed_log_weight = log_weights[n]
        log_accept_prob = proposed_log_weight - current_log_weight

        if (torch.log(torch.rand(1)) < log_accept_prob):
            current_trajectory = proposed_trajectory.clone()
            current_log_weight = proposed_log_weight.clone()  
            num_accept += 1
    accept_rate = num_accept / N
    BDB['acceptrate'][r] = accept_rate

    # print
    print('Repeat: ' + str(r) + 
          ' ESS%: ' + str(float(ess * 100 / N)) + 
          ' log-transition density: ' + str(float(log_transition_estimate)), 
          ' Accept rate: ' + str(float(accept_rate)))

# simulate modified backward diffusion bridge (MBDB) process with approximate score
MBDB = {'ess' : torch.zeros(R), 'logestimate' : torch.zeros(R), 'acceptrate' : torch.zeros(R)}
for r in range(R):
    with torch.no_grad():
        output = diffusion.simulate_bridge_backwards(score_transition_net, X0, XT, epsilon, N, modify = True)
        trajectories = output['trajectories']
        log_proposal = output['logdensity']
    log_target = diffusion.law_bridge(trajectories) 
    log_weights = log_target - log_proposal

    # importance sampling
    max_log_weights = torch.max(log_weights)
    weights = torch.exp(log_weights - max_log_weights)
    norm_weights = weights / torch.sum(weights)
    ess = 1.0 / torch.sum(norm_weights**2)
    log_transition_estimate = torch.log(torch.mean(weights)) + max_log_weights
    MBDB['ess'][r] = ess
    MBDB['logestimate'][r] = log_transition_estimate

    # independent Metropolis-Hastings
    initial = diffusion.simulate_bridge_backwards(score_transition_net, X0, XT, epsilon, 1, modify = True)
    current_trajectory = initial['trajectories']
    current_log_proposal = initial['logdensity'] 
    current_log_target = diffusion.law_bridge(current_trajectory)
    current_log_weight = current_log_target - current_log_proposal
    num_accept = 0
    for n in range(N):
        proposed_trajectory = trajectories[n, :, :]
        proposed_log_weight = log_weights[n]
        log_accept_prob = proposed_log_weight - current_log_weight

        if (torch.log(torch.rand(1)) < log_accept_prob):
            current_trajectory = proposed_trajectory.clone()
            current_log_weight = proposed_log_weight.clone()  
            num_accept += 1
    accept_rate = num_accept / N
    MBDB['acceptrate'][r] = accept_rate

    # print
    print('Repeat: ' + str(r) + 
          ' ESS%: ' + str(float(ess * 100 / N)) + 
          ' log-transition density: ' + str(float(log_transition_estimate)),
          ' Accept rate: ' + str(float(accept_rate)))

Optimization iteration: 1 Loss: 50.43480682373047
Optimization iteration: 50 Loss: 42.75078201293945
Optimization iteration: 100 Loss: 42.617889404296875
Optimization iteration: 150 Loss: 42.62534713745117
Optimization iteration: 200 Loss: 43.61304473876953
Optimization iteration: 250 Loss: 44.4798469543457
Optimization iteration: 300 Loss: 42.157230377197266
Optimization iteration: 350 Loss: 43.87175750732422
Optimization iteration: 400 Loss: 42.97872543334961
Optimization iteration: 450 Loss: 43.07283020019531
Optimization iteration: 500 Loss: 42.22943115234375
Repeat: 0 ESS%: 77.22417449951172 log-transition density: -1.718825340270996  Accept rate: 0.6923828125
Repeat: 1 ESS%: 74.99894714355469 log-transition density: -1.730238437652588  Accept rate: 0.6669921875
Repeat: 2 ESS%: 75.43958282470703 log-transition density: -1.7491873502731323  Accept rate: 0.671875
Repeat: 3 ESS%: 76.96150970458984 log-transition density: -1.7112891674041748  Accept rate: 0.6875
Repeat: 4 ESS%: 76.174

In [29]:
# learn forward diffusion bridge process with score matching
epsilon = 1e-3
minibatch = 100
num_iterations = 500
learning_rate = 0.01
ema_momentum = 0.99
output = diffusion.learn_score_marginal(score_transition_net, X0, XT, epsilon, minibatch, num_iterations, learning_rate, ema_momentum)
score_marginal_net = output['net']

# simulate forward diffusion bridge (FDB) process using approximate score
FDB = {'ess' : torch.zeros(R), 'logestimate' : torch.zeros(R), 'acceptrate' : torch.zeros(R)}
for r in range(R):
    with torch.no_grad():
        output = diffusion.simulate_bridge_forwards(score_transition_net, score_marginal_net, X0, XT, epsilon, N, modify = False)
        trajectories = output['trajectories']
        log_proposal = output['logdensity']
    log_target = diffusion.law_bridge(trajectories) 
    log_weights = log_target - log_proposal

    # importance sampling
    max_log_weights = torch.max(log_weights)
    weights = torch.exp(log_weights - max_log_weights)
    norm_weights = weights / torch.sum(weights)
    ess = 1.0 / torch.sum(norm_weights**2)
    log_transition_estimate = torch.log(torch.mean(weights)) + max_log_weights
    FDB['ess'][r] = ess
    FDB['logestimate'][r] = log_transition_estimate

    # independent Metropolis-Hastings
    initial = diffusion.simulate_bridge_forwards(score_transition_net, score_marginal_net, X0, XT, epsilon, 1, modify = False)
    current_trajectory = initial['trajectories']
    current_log_proposal = initial['logdensity'] 
    current_log_target = diffusion.law_bridge(current_trajectory)
    current_log_weight = current_log_target - current_log_proposal
    num_accept = 0
    for n in range(N):
        proposed_trajectory = trajectories[n, :, :]
        proposed_log_weight = log_weights[n]
        log_accept_prob = proposed_log_weight - current_log_weight

        if (torch.log(torch.rand(1)) < log_accept_prob):
            current_trajectory = proposed_trajectory.clone()
            current_log_weight = proposed_log_weight.clone()  
            num_accept += 1
    accept_rate = num_accept / N
    FDB['acceptrate'][r] = accept_rate

    # print
    print('Repeat: ' + str(r) + 
          ' ESS%: ' + str(float(ess * 100 / N)) + 
          ' log-transition density: ' + str(float(log_transition_estimate)),
          ' Accept rate: ' + str(float(accept_rate)))
          
# simulate modified forward diffusion bridge (MFDB) process using approximate score
MFDB = {'ess' : torch.zeros(R), 'logestimate' : torch.zeros(R), 'acceptrate' : torch.zeros(R)}
for r in range(R):
    with torch.no_grad():
        output = diffusion.simulate_bridge_forwards(score_transition_net, score_marginal_net, X0, XT, epsilon, N, modify = True)
        trajectories = output['trajectories']
        log_proposal = output['logdensity']
    log_target = diffusion.law_bridge(trajectories) 
    log_weights = log_target - log_proposal

    # importance sampling
    max_log_weights = torch.max(log_weights)
    weights = torch.exp(log_weights - max_log_weights)
    norm_weights = weights / torch.sum(weights)
    ess = 1.0 / torch.sum(norm_weights**2)
    log_transition_estimate = torch.log(torch.mean(weights)) + max_log_weights
    MFDB['ess'][r] = ess
    MFDB['logestimate'][r] = log_transition_estimate

    # independent Metropolis-Hastings
    initial = diffusion.simulate_bridge_forwards(score_transition_net, score_marginal_net, X0, XT, epsilon, 1, modify = True)
    current_trajectory = initial['trajectories']
    current_log_proposal = initial['logdensity'] 
    current_log_target = diffusion.law_bridge(current_trajectory)
    current_log_weight = current_log_target - current_log_proposal
    num_accept = 0
    for n in range(N):
        proposed_trajectory = trajectories[n, :, :]
        proposed_log_weight = log_weights[n]
        log_accept_prob = proposed_log_weight - current_log_weight

        if (torch.log(torch.rand(1)) < log_accept_prob):
            current_trajectory = proposed_trajectory.clone()
            current_log_weight = proposed_log_weight.clone()  
            num_accept += 1
    accept_rate = num_accept / N
    MFDB['acceptrate'][r] = accept_rate

    # print
    print('Repeat: ' + str(r) + 
          ' ESS%: ' + str(float(ess * 100 / N)) + 
          ' log-transition density: ' + str(float(log_transition_estimate)),
          ' Accept rate: ' + str(float(accept_rate)))
        

Optimization iteration: 1 Loss: 53.575618743896484
Optimization iteration: 50 Loss: 43.43586349487305
Optimization iteration: 100 Loss: 45.451759338378906
Optimization iteration: 150 Loss: 45.930606842041016
Optimization iteration: 200 Loss: 45.80348587036133
Optimization iteration: 250 Loss: 44.68633270263672
Optimization iteration: 300 Loss: 45.43576431274414
Optimization iteration: 350 Loss: 44.035560607910156
Optimization iteration: 400 Loss: 43.07563018798828
Optimization iteration: 450 Loss: 42.61347579956055
Optimization iteration: 500 Loss: 44.18614196777344
Repeat: 0 ESS%: 72.89212799072266 log-transition density: -1.7556531429290771  Accept rate: 0.685546875
Repeat: 1 ESS%: 72.98921966552734 log-transition density: -1.7718067169189453  Accept rate: 0.658203125
Repeat: 2 ESS%: 73.33185577392578 log-transition density: -1.723029613494873  Accept rate: 0.697265625
Repeat: 3 ESS%: 74.34981536865234 log-transition density: -1.7237424850463867  Accept rate: 0.6630859375
Repeat: 4 E

In [30]:
# forward diffusion (FD) method of Pedersen (1995)
drift = f
FD = {'ess' : torch.zeros(R), 'logestimate' : torch.zeros(R), 'acceptrate' : torch.zeros(R)}

for r in range(R):
    output = diffusion.simulate_proposal_bridge(drift, X0, XT, N)
    trajectories = output['trajectories']
    log_proposal = output['logdensity']
    log_target = diffusion.law_bridge(trajectories) 
    log_weights = log_target - log_proposal

    # importance sampling
    max_log_weights = torch.max(log_weights)
    weights = torch.exp(log_weights - max_log_weights)
    norm_weights = weights / torch.sum(weights)
    ess = 1.0 / torch.sum(norm_weights**2)    
    log_transition_estimate = torch.log(torch.mean(weights)) + max_log_weights
    FD['ess'][r] = ess
    FD['logestimate'][r] = log_transition_estimate

    # independent Metropolis-Hastings
    initial = diffusion.simulate_proposal_bridge(drift, X0, XT, 1)
    current_trajectory = initial['trajectories']
    current_log_proposal = initial['logdensity'] 
    current_log_target = diffusion.law_bridge(current_trajectory)
    current_log_weight = current_log_target - current_log_proposal
    num_accept = 0
    for n in range(N):
        proposed_trajectory = trajectories[n, :, :]
        proposed_log_weight = log_weights[n]
        log_accept_prob = proposed_log_weight - current_log_weight

        if (torch.log(torch.rand(1)) < log_accept_prob):
            current_trajectory = proposed_trajectory.clone()
            current_log_weight = proposed_log_weight.clone()  
            num_accept += 1
    accept_rate = num_accept / N
    FD['acceptrate'][r] = accept_rate
    
    # print
    print('Repeat: ' + str(r) + 
          ' ESS%: ' + str(float(ess * 100 / N)) + 
          ' log-transition density: ' + str(float(log_transition_estimate)),
          ' Accept rate: ' + str(float(accept_rate)))


Repeat: 0 ESS%: 8.323308944702148 log-transition density: -1.9021012783050537  Accept rate: 0.08984375
Repeat: 1 ESS%: 9.887975692749023 log-transition density: -1.6425397396087646  Accept rate: 0.09765625
Repeat: 2 ESS%: 11.170169830322266 log-transition density: -1.5253095626831055  Accept rate: 0.087890625
Repeat: 3 ESS%: 8.123613357543945 log-transition density: -1.8484628200531006  Accept rate: 0.0732421875
Repeat: 4 ESS%: 8.784488677978516 log-transition density: -1.8412363529205322  Accept rate: 0.0966796875
Repeat: 5 ESS%: 9.558592796325684 log-transition density: -1.764056921005249  Accept rate: 0.103515625
Repeat: 6 ESS%: 8.64162826538086 log-transition density: -1.770876407623291  Accept rate: 0.09375
Repeat: 7 ESS%: 9.18686580657959 log-transition density: -1.7537717819213867  Accept rate: 0.0830078125
Repeat: 8 ESS%: 9.36159610748291 log-transition density: -1.6771519184112549  Accept rate: 0.087890625
Repeat: 9 ESS%: 8.258002281188965 log-transition density: -1.8993415832

In [31]:
# modified diffusion bridge (MDB) method of Durham and Gallant (2002)
drift = lambda t,x: (XT - x) / (T - t)
MDB = {'ess' : torch.zeros(R), 'logestimate' : torch.zeros(R), 'acceptrate' : torch.zeros(R)}

for r in range(R):
    output = diffusion.simulate_proposal_bridge(drift, X0, XT, N, modify = True)
    trajectories = output['trajectories']
    log_proposal = output['logdensity']
    log_target = diffusion.law_bridge(trajectories) 
    log_weights = log_target - log_proposal
    
    # importance sampling
    max_log_weights = torch.max(log_weights)
    weights = torch.exp(log_weights - max_log_weights)
    norm_weights = weights / torch.sum(weights)
    ess = 1.0 / torch.sum(norm_weights**2)
    log_transition_estimate = torch.log(torch.mean(weights)) + max_log_weights
    MDB['ess'] = ess
    MDB['logestimate'] = log_transition_estimate

    # independent Metropolis-Hastings
    initial = diffusion.simulate_proposal_bridge(drift, X0, XT, 1, modify = True)
    current_trajectory = initial['trajectories']
    current_log_proposal = initial['logdensity'] 
    current_log_target = diffusion.law_bridge(current_trajectory)
    current_log_weight = current_log_target - current_log_proposal
    num_accept = 0
    for n in range(N):
        proposed_trajectory = trajectories[n, :, :]
        proposed_log_weight = log_weights[n]
        log_accept_prob = proposed_log_weight - current_log_weight

        if (torch.log(torch.rand(1)) < log_accept_prob):
            current_trajectory = proposed_trajectory.clone()
            current_log_weight = proposed_log_weight.clone()  
            num_accept += 1
    accept_rate = num_accept / N
    MDB['acceptrate'][r] = accept_rate

    # print
    print('Repeat: ' + str(r) + 
          ' ESS%: ' + str(float(ess * 100 / N)) + 
          ' log-transition density: ' + str(float(log_transition_estimate)),
          ' Accept rate: ' + str(float(accept_rate)))

Repeat: 0 ESS%: 50.77891540527344 log-transition density: -1.7116484642028809  Accept rate: 0.498046875
Repeat: 1 ESS%: 52.78487777709961 log-transition density: -1.7449495792388916  Accept rate: 0.4609375
Repeat: 2 ESS%: 49.96184539794922 log-transition density: -1.709021806716919  Accept rate: 0.501953125
Repeat: 3 ESS%: 52.70448303222656 log-transition density: -1.7707666158676147  Accept rate: 0.4775390625
Repeat: 4 ESS%: 51.389320373535156 log-transition density: -1.7212423086166382  Accept rate: 0.498046875
Repeat: 5 ESS%: 53.712100982666016 log-transition density: -1.7651145458221436  Accept rate: 0.505859375
Repeat: 6 ESS%: 53.7191276550293 log-transition density: -1.7497261762619019  Accept rate: 0.537109375
Repeat: 7 ESS%: 52.26784896850586 log-transition density: -1.7277072668075562  Accept rate: 0.5087890625
Repeat: 8 ESS%: 54.534358978271484 log-transition density: -1.7553759813308716  Accept rate: 0.5380859375
Repeat: 9 ESS%: 53.2274055480957 log-transition density: -1.68

In [32]:
# diffusion bridge proposal of Clark (1990) and Delyon and Hu (2006) (CDH)
drift = lambda t,x: f(t,x) + (XT - x) / (T - t)
CDH = {'ess' : torch.zeros(R), 'logestimate' : torch.zeros(R), 'acceptrate' : torch.zeros(R)}

for r in range(R):
    output = diffusion.simulate_proposal_bridge(drift, X0, XT, N, modify = False)
    trajectories = output['trajectories']
    log_proposal = output['logdensity']
    log_target = diffusion.law_bridge(trajectories) 
    log_weights = log_target - log_proposal

    # importance sampling
    max_log_weights = torch.max(log_weights)
    weights = torch.exp(log_weights - max_log_weights)
    norm_weights = weights / torch.sum(weights)
    ess = 1.0 / torch.sum(norm_weights**2)
    log_transition_estimate = torch.log(torch.mean(weights)) + max_log_weights
    CDH['ess'] = ess
    CDH['logestimate'] = log_transition_estimate

    # independent Metropolis-Hastings
    initial = diffusion.simulate_proposal_bridge(drift, X0, XT, 1, modify = False)
    current_trajectory = initial['trajectories']
    current_log_proposal = initial['logdensity'] 
    current_log_target = diffusion.law_bridge(current_trajectory)
    current_log_weight = current_log_target - current_log_proposal
    num_accept = 0
    for n in range(N):
        proposed_trajectory = trajectories[n, :, :]
        proposed_log_weight = log_weights[n]
        log_accept_prob = proposed_log_weight - current_log_weight

        if (torch.log(torch.rand(1)) < log_accept_prob):
            current_trajectory = proposed_trajectory.clone()
            current_log_weight = proposed_log_weight.clone()  
            num_accept += 1
    accept_rate = num_accept / N
    CDH['acceptrate'][r] = accept_rate

    # print
    print('Repeat: ' + str(r) + 
          ' ESS%: ' + str(float(ess * 100 / N)) + 
          ' log-transition density: ' + str(float(log_transition_estimate)),
          ' Accept rate: ' + str(float(accept_rate)))

Repeat: 0 ESS%: 33.57039260864258 log-transition density: -1.7434618473052979  Accept rate: 0.5068359375
Repeat: 1 ESS%: 26.16057014465332 log-transition density: -1.7571601867675781  Accept rate: 0.5107421875
Repeat: 2 ESS%: 29.68663215637207 log-transition density: -1.7389836311340332  Accept rate: 0.4462890625
Repeat: 3 ESS%: 24.4841251373291 log-transition density: -1.7528324127197266  Accept rate: 0.4580078125
Repeat: 4 ESS%: 31.571802139282227 log-transition density: -1.7408638000488281  Accept rate: 0.4853515625
Repeat: 5 ESS%: 40.510650634765625 log-transition density: -1.8191678524017334  Accept rate: 0.501953125
Repeat: 6 ESS%: 26.43208885192871 log-transition density: -1.755610704421997  Accept rate: 0.4326171875
Repeat: 7 ESS%: 24.115394592285156 log-transition density: -1.7506752014160156  Accept rate: 0.412109375
Repeat: 8 ESS%: 25.561368942260742 log-transition density: -1.8060674667358398  Accept rate: 0.490234375
Repeat: 9 ESS%: 28.697416305541992 log-transition densit

In [36]:
# compare ESS
print('FD ESS%: ' + str(float(torch.mean(FD['ess']) * 100 / N)))
print('MDB ESS%: ' + str(float(torch.mean(MDB['ess']) * 100 / N)))
print('CDH ESS%: ' + str(float(torch.mean(CDH['ess']) * 100 / N)))
print('BDB ESS%: ' + str(float(torch.mean(BDB['ess']) * 100 / N)))
print('FDB ESS%: ' + str(float(torch.mean(FDB['ess']) * 100 / N)))
print('MBDB ESS%: ' + str(float(torch.mean(MBDB['ess']) * 100 / N)))
print('MFDB ESS%: ' + str(float(torch.mean(MFDB['ess']) * 100 / N)))


FD ESS%: 9.170830726623535
MDB ESS%: 52.73068618774414
CDH ESS%: 31.60641860961914
BDB ESS%: 75.97206115722656
FDB ESS%: 72.84327697753906
MBDB ESS%: 93.87237548828125
MFDB ESS%: 90.37039184570312


In [34]:
# compare RMSE of log-transition density 
print('FD RMSE: ' + str(float(torch.sqrt(torch.mean((FD['logestimate'] - log_transition_density)**2)))))
print('MDB RMSE: ' + str(float(torch.sqrt(torch.mean((MDB['logestimate'] - log_transition_density)**2)))))
print('CDH RMSE: ' + str(float(torch.sqrt(torch.mean((CDH['logestimate'] - log_transition_density)**2)))))
print('BDB RMSE: ' + str(float(torch.sqrt(torch.mean((BDB['logestimate'] - log_transition_density)**2)))))
print('FDB RMSE: ' + str(float(torch.sqrt(torch.mean((FDB['logestimate'] - log_transition_density)**2)))))
print('MBDB RMSE: ' + str(float(torch.sqrt(torch.mean((MBDB['logestimate'] - log_transition_density)**2)))))
print('MFDB RMSE: ' + str(float(torch.sqrt(torch.mean((MFDB['logestimate'] - log_transition_density)**2)))))

FD RMSE: 0.09557279944419861
MDB RMSE: 0.03220629692077637
CDH RMSE: 0.02803778648376465
BDB RMSE: 0.015261337161064148
FDB RMSE: 0.018894940614700317
MBDB RMSE: 0.008424938656389713
MFDB RMSE: 0.01014381181448698


In [38]:
# compare indepedent Meteropolis-Hastings acceptance rate
print('FD acceptance%: ' + str(float(torch.mean(FD['acceptrate'] * 100))))
print('MDB acceptance%: ' + str(float(torch.mean(MDB['acceptrate'] * 100))))
print('CDH acceptance%: ' + str(float(torch.mean(CDH['acceptrate'] * 100))))
print('BDB acceptance%: ' + str(float(torch.mean(BDB['acceptrate'] * 100))))
print('FDB acceptance%: ' + str(float(torch.mean(FDB['acceptrate'] * 100))))
print('MBDB acceptance%: ' + str(float(torch.mean(MBDB['acceptrate'] * 100))))
print('MFDB acceptance%: ' + str(float(torch.mean(MFDB['acceptrate'] * 100))))


FD acceptance%: 8.5859375
MDB acceptance%: 51.4189453125
CDH acceptance%: 46.369140625
BDB acceptance%: 67.7578125
FDB acceptance%: 65.9130859375
MBDB acceptance%: 86.3984375
MFDB acceptance%: 82.4462890625
